Parse whatsapp chat export between alex and yashodan

In [ ]:
from datetime import datetime
import pandas as pd
pd.set_option('display.max_colwidth', 800)

In [ ]:
df = pd.read_csv('data_original.txt', delimiter = "\n")

In [ ]:
df.columns = ['message']

In [ ]:
def get_author(x):
    if '- Alex Conway:' in x:
        return 'alex'
    else:
        return 'yashodan'

In [ ]:
def get_content(x):
    return ' '.join(x.split(':')[2:]).strip()

In [ ]:
def scrub_media(x):
    return x.replace("<Media omitted>", "")

In [ ]:
def scrub_urls(x):
    return x if 'http' not in x else  ''

In [ ]:
def get_ts(x):
    return datetime.strptime(x.split(" -")[0].replace(',',''), '%Y/%m/%d %H:%M %p')

In [ ]:
df['author'] = df['message'].apply(lambda x: get_author(x))

In [ ]:
df['content'] = df['message'].apply(lambda x: get_content(x))

In [ ]:
df['content'] = df['content'].apply(lambda x: scrub_media(x))

In [ ]:
df['content'] = df['content'].apply(lambda x: scrub_urls(x))

In [ ]:
df['content_len'] = df['content'].str.len()

In [ ]:
df = df[df['content_len'] > 0]

In [ ]:
df['ts'] = df['message'].apply(lambda x: get_ts(x))
df['ts'] = pd.to_datetime(df['ts'])

In [ ]:
df

In [ ]:
df['replying_to_author'] = df['author'].shift(1)
df['replying_to'] = df['content'].shift(1)

In [ ]:
df = df.dropna()

In [ ]:
df.head(20)

In [ ]:
df['is_reply'] = df['replying_to_author'] != df['author']

In [ ]:
df = df[df['is_reply'] == 1]

In [ ]:
!tail -20 'data_original.txt'

In [ ]:
df.head(20)

In [ ]:
df_a = df[df['author'] == 'alex'][['content','replying_to']]
df_a.columns = ['completion', 'prompt']
df_a.to_csv('data_alex.csv', index=False)
data_a = list(df_a.to_dict(orient='index').values())

In [ ]:
df_b = df[df['author'] == 'yashodan'][['content','replying_to']]
df_b.columns = ['completion', 'prompt']
df_b.to_csv('data_yashodan.csv', index=False)
data_b = list(df_b.to_dict(orient='index').values())

In [ ]:
with open('data_alex.jsonl', 'w') as f:
    for item in data_a:
        f.write("%s\n" % item)

In [ ]:
with open('data_yashodan.jsonl', 'w') as f:
    for item in data_b:
        f.write("%s\n" % item)